In [1]:
!pip install -q transformers datasets evaluate bert_score nltk
!pip install -q git+https://github.com/google-research/bleurt.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

In [2]:
# ================================
# Essential Libraries
# ================================
import os
from tqdm import tqdm
import torch
import torch.nn as nn

# ================================
# Hugging Face Transformers & Datasets
# ================================
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    T5Tokenizer,
    pipeline,
    
)
from torch.optim import AdamW

from datasets import load_from_disk
from torch.utils.data import Dataset, DataLoader

# ================================
# Evaluation & Metrics
# ================================
import evaluate
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score as bert_score

# ================================
# NLTK Setup (if not downloaded)
# ================================
import nltk
nltk.download('punkt')


2025-04-11 13:49:22.948464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744379363.178425      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744379363.249464      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Dataset Paths
dataset_path = "/kaggle/input/datasetnut/conv_data"
train_dataset = load_from_disk(os.path.join(dataset_path, "therapy_train"))
val_dataset = load_from_disk(os.path.join(dataset_path, "therapy_val"))
test_dataset = load_from_disk(os.path.join(dataset_path, "therapy_test"))

In [4]:
# ============================================
# Part 2: Dataset Class for Tokenization
# ============================================
class ConversationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        input_text = "predict next utterance: " + sample["input_text"]
        target_text = sample["target_text"]
        inputs = self.tokenizer(input_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        targets = self.tokenizer(target_text, max_length=50, padding="max_length", truncation=True, return_tensors="pt")
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": targets["input_ids"].squeeze()
        }

In [5]:
# ============================================
# Part 3: Load Main Models & Tokenizers
# ============================================
model1_name = "google/mt5-small"
model2_name = "google/flan-t5-base"

model1 = AutoModelForSeq2SeqLM.from_pretrained(model1_name)
tokenizer1 = AutoTokenizer.from_pretrained(model1_name)

model2 = AutoModelForSeq2SeqLM.from_pretrained(model2_name)
tokenizer2 = AutoTokenizer.from_pretrained(model2_name)

rewriter_model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-small-finetuned-emotion")
rewriter_tokenizer = T5Tokenizer.from_pretrained("mrm8488/t5-small-finetuned-emotion")

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [6]:
# ============================================
# Part 4: Sentiment Detection & Alignment
# ============================================
sentiment_classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def get_sentiment(text, max_length=512):
    # truncate input to max token limit
    truncated_text = text[:max_length]
    result = sentiment_classifier(truncated_text)[0]["label"].lower()
    if "positive" in result:
        return 2
    elif "negative" in result:
        return 0
    else:
        return 1


def align_sentiment(prediction, target_sentiment):
    pred_sentiment = get_sentiment(prediction)
    if pred_sentiment == target_sentiment:
        return prediction
    sentiment_map = {0: "negative", 1: "neutral", 2: "positive"}
    prompt = f"rephrase to be {sentiment_map[target_sentiment]}: {prediction}"
    input_ids = rewriter_tokenizer(prompt, return_tensors="pt").input_ids.to(rewriter_model.device)
    output_ids = rewriter_model.generate(input_ids, max_length=60, num_beams=5)
    return rewriter_tokenizer.decode(output_ids[0], skip_special_tokens=True)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
# ============================================
# Part 5: Fusion Model with LSTM/GRU Support
# ============================================
class FusionModel(nn.Module):
    def __init__(self, base_model, fusion="lstm"):
        super(FusionModel, self).__init__()
        self.base_model = base_model
        hidden_size = base_model.config.d_model

        if fusion == "lstm":
            self.fusion_layer = nn.LSTM(hidden_size, hidden_size, batch_first=True, bidirectional=True)
            self.linear = nn.Linear(hidden_size * 2, hidden_size)
        elif fusion == "gru":
            self.fusion_layer = nn.GRU(hidden_size, hidden_size, batch_first=True, bidirectional=True)
            self.linear = nn.Linear(hidden_size * 2, hidden_size)
        elif fusion == "lstm_gru":
            self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True, bidirectional=True)
            self.gru = nn.GRU(hidden_size * 2, hidden_size, batch_first=True, bidirectional=True)
            self.linear = nn.Linear(hidden_size * 2, hidden_size)
            self.use_double = True
        else:
            raise ValueError("Unsupported fusion type")

        self.use_double = fusion == "lstm_gru"

    def forward(self, input_ids, attention_mask, labels=None):
        encoder = self.base_model.get_encoder()
        encoder_outputs = encoder(input_ids=input_ids, attention_mask=attention_mask)

        if self.use_double:
            x, _ = self.lstm(encoder_outputs.last_hidden_state)
            x, _ = self.gru(x)
        else:
            x, _ = self.fusion_layer(encoder_outputs.last_hidden_state)

        reduced_output = self.linear(x)

        outputs = self.base_model(
            inputs_embeds=reduced_output,
            attention_mask=attention_mask,
            labels=labels,
        )
        return outputs

In [8]:
# ============================================
# Part 6: Training Function
# ============================================
def train_model(model, tokenizer, train_dataset, val_dataset, epochs=10, batch_size=4, lr=5e-5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    train_loader = DataLoader(ConversationDataset(train_dataset, tokenizer), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(ConversationDataset(val_dataset, tokenizer), batch_size=batch_size)
    optimizer = Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1} Training Loss: {total_loss/len(train_loader):.4f}")

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += outputs.loss.item()

        print(f"Epoch {epoch+1} Validation Loss: {val_loss/len(val_loader):.4f}")

    return model


In [9]:
def generate_response(model, tokenizer, input_text, target_sentiment):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    input_text = "predict next utterance: " + input_text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(device)

    if hasattr(model, "generate"):  # for base T5/MT5 models
        outputs = model.generate(inputs["input_ids"], max_length=50, num_beams=5)
    else:  # for FusionModel
        with torch.no_grad():
            input_ids = inputs["input_ids"]
            attention_mask = inputs["attention_mask"]

            encoder = model.base_model.get_encoder()
            encoder_outputs = encoder(input_ids=input_ids, attention_mask=attention_mask)

            if model.use_double:
                x, _ = model.lstm(encoder_outputs.last_hidden_state)
                x, _ = model.gru(x)
            else:
                x, _ = model.fusion_layer(encoder_outputs.last_hidden_state)

            reduced_output = model.linear(x)

            decoder_input_ids = torch.tensor([[tokenizer.pad_token_id]]).to(device)

            generated_ids = []
            for _ in range(50):
                outputs = model.base_model(
                    inputs_embeds=reduced_output,
                    attention_mask=attention_mask,
                    decoder_input_ids=decoder_input_ids,
                )
                next_token_logits = outputs.logits[:, -1, :]
                next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
                decoder_input_ids = torch.cat([decoder_input_ids, next_token_id], dim=-1)
                if next_token_id.item() == tokenizer.eos_token_id:
                    break
                generated_ids.append(next_token_id.item())

            outputs = torch.tensor(generated_ids).unsqueeze(0).to(device)

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return align_sentiment(prediction, target_sentiment)



In [10]:
!pip install -q rouge_score


  Preparing metadata (setup.py) ... done


In [11]:
# ============================================
# Part 8: Evaluation
# ============================================
bleurt = evaluate.load("bleurt", checkpoint="bleurt-base-128")
rouge = evaluate.load("rouge")

def compute_metrics(preds, refs):
    bleu_scores = [sentence_bleu([ref.split()], pred.split(), smoothing_function=SmoothingFunction().method1) for pred, ref in zip(preds, refs)]
    P, R, F1 = bert_score(preds, refs, lang="en", verbose=False)
    rouge_result = rouge.compute(predictions=preds, references=refs)
    bleurt_score = bleurt.compute(predictions=preds, references=refs)["scores"]

    return {
        "BLEU": sum(bleu_scores)/len(bleu_scores),
        "ROUGE-1": rouge_result["rouge1"],
        "ROUGE-2": rouge_result["rouge2"],
        "ROUGE-L": rouge_result["rougeL"],
        "BERTScore F1": float(F1.mean()),
        "BLEURT": sum(bleurt_score)/len(bleurt_score),
    }

I0000 00:00:1744379436.496790      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15221 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [12]:
from torch.optim import Adam


In [13]:
# ============================================
# Part 9: Model Training & Evaluation
# ============================================
model1 = train_model(model1, tokenizer1, train_dataset, val_dataset)
model2 = train_model(model2, tokenizer2, train_dataset, val_dataset)
fusion_m3 = train_model(FusionModel(model1, fusion="lstm_gru"), tokenizer1, train_dataset, val_dataset)
fusion_m4 = train_model(FusionModel(model2, fusion="lstm_gru"), tokenizer2, train_dataset, val_dataset)
refs = [test_dataset[i]["target_text"] for i in range(len(test_dataset))]

Training Epoch 1: 100%|██████████| 1002/1002 [02:51<00:00,  5.85it/s]


Epoch 1 Training Loss: 14.5460
Epoch 1 Validation Loss: 3.6904


Training Epoch 2: 100%|██████████| 1002/1002 [02:50<00:00,  5.89it/s]


Epoch 2 Training Loss: 3.9905
Epoch 2 Validation Loss: 2.8317


Training Epoch 3: 100%|██████████| 1002/1002 [02:49<00:00,  5.90it/s]


...

Epoch 8 Training Loss: 1.0212
Epoch 8 Validation Loss: 1.6439


Training Epoch 9: 100%|██████████| 1002/1002 [12:58<00:00,  1.29it/s]


Epoch 9 Training Loss: 0.9907
Epoch 9 Validation Loss: 1.6534


Training Epoch 10: 100%|██████████| 1002/1002 [12:58<00:00,  1.29it/s]


Epoch 10 Training Loss: 0.9637
Epoch 10 Validation Loss: 1.6773


In [15]:
# ============================================
# Part 10: Generate and Evaluate All Models
# ============================================

def evaluate_model_on_test(model, tokenizer, test_dataset):
    preds = []
    for sample in tqdm(test_dataset, desc="Generating predictions"):
        input_text = sample["input_text"]
        target_text = sample["target_text"]
        target_sentiment = get_sentiment(target_text)
        response = generate_response(model, tokenizer, input_text, target_sentiment)
        preds.append(response)
    return compute_metrics(preds, refs)

# Evaluate all models
results = {}

results["Model1 (MT5)"] = evaluate_model_on_test(model1, tokenizer1, test_dataset)
results["Model2 (T5)"] = evaluate_model_on_test(model2, tokenizer2, test_dataset)
results["Model1 + LSTM+GRU"] = evaluate_model_on_test(fusion_m3, tokenizer1, test_dataset)
results["Model2 + LSTM+GRU"] = evaluate_model_on_test(fusion_m4, tokenizer2, test_dataset)

# Display results
import pandas as pd
results_df = pd.DataFrame(results).T
print("\nEvaluation Results:")
print(results_df)


Generating predictions: 100%|██████████| 968/968 [12:23<00:00,  1.30it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Generating predictions: 100%|██████████| 968/968 [14:54<00:00,  1.08it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Generating predictions: 100%|██████████| 968/968 [20:02<00:00,  1.24s/it]  
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task 


Evaluation Results:
                       BLEU   ROUGE-1   ROUGE-2   ROUGE-L  BERTScore F1  \
Model1 (MT5)       0.001159  0.025705  0.003484  0.023650      0.819328   
Model2 (T5)        0.005368  0.072331  0.014320  0.064172      0.822860   
Model1 + LSTM+GRU  0.002796  0.053783  0.009155  0.047499      0.824957   
Model2 + LSTM+GRU  0.002103  0.050307  0.005548  0.045264      0.814564   

                     BLEURT  
Model1 (MT5)      -1.356782  
Model2 (T5)       -1.322900  
Model1 + LSTM+GRU -1.376222  
Model2 + LSTM+GRU -1.385983  
